In [37]:
#Dependencies 
import requests
import json
import pandas as pd
from api_key import yelp_api_key, mapbox_api_key
# C:\Users\nwf91\OneDrive\Desktop\VandyDataCourse\project-3\project-3\api_key.py
# C:\Users\nwf91\OneDrive\Desktop\VandyDataCourse\project-3\project-3\working_doc.ipynb

from pprint import pprint

In [38]:
# Test if the API keys are imported correctly
print(f"Yelp API Key: {yelp_api_key}")
print(f"Mapbox API Key: {mapbox_api_key}")

Yelp API Key: GfAgyvCCqzBJJrefnJOUX_fcJI02oK4PgB2mMkTwNmWSXDEPhJPU9kYctePGP49D0ujK3QqYE4_Iic-RfsWG_ULgbOWv8yx1jRUrgVpj50Vq6qws4Y1Nkn-n2_TgZnYx
Mapbox API Key: pk.eyJ1IjoibmJpb3JnOTkiLCJhIjoiY20weDd5OG43MDFxbDJzcHBwN2Y2aXUxYiJ9.U3TN-LcnCc7RnK25R74N5w


In [47]:
# Yelp API 

# Yelp API base URL for searching businesses
yelp_url = "https://api.yelp.com/v3/businesses/search"

# Location and other parameters
location = "san francisco"
term = "restaurants"  
limit = 40 # Maximum results per request 
offset = 0 
rounds = 6 

yelp_data = []

for round in range(rounds): 

    # Define the headers to include the API key
    headers = {
        "Authorization": f"Bearer {yelp_api_key}"
    }

    # Define the parameters for the API call
    yelp_params = {
        "location": location,
        "term": term,
        "limit": limit,
        "offset": offset 
    }

    # Make the GET request to the Yelp API
    yelp_response = requests.get(yelp_url, headers=headers, params=yelp_params)
        
    #Saving Yelp Data 
    # Check if the Yelp request was successful
    if yelp_response.status_code == 200:
        yelp_data_json = yelp_response.json()  # Convert the response to JSON format
        businesses = yelp_data_json.get("businesses", [])  # Extract the businesses from the response
        

        # Loop over the Yelp businesses and call WalkScore API for each business
        for business in businesses:
            business_name = business["name"]
            business_id = business["id"]
            latitude = business["coordinates"]["latitude"]
            longitude = business["coordinates"]["longitude"]
            address = ", ".join(business["location"]["display_address"])
            rating = business["rating"]
            review_count = business["review_count"]
            price = business.get("price", "N/A")  # Some businesses may not have a price listed
            bus_categories = ", ".join([category["title"] for category in business["categories"]]),
            transactions = business["transactions"]
            
            yelp_data.append({
                "Name" : business_name,
                "Business_ID": business_id,
                "Latitude": latitude, 
                "Longitude": longitude, 
                "Address": address,
                "Rating": rating,
                "Review_count": review_count,
                "Price": price, 
                "Categories":bus_categories,
                "Transactions": transactions
            })
        
    else:
        print(f"Error: {yelp_response.status_code} - {yelp_response.text}") 

    offset += 40

yelp_pd = pd.DataFrame(yelp_data)
yelp_pd


,Name,Business_ID,Latitude,Longitude,Address,Rating,Review_count,Price,Categories,Transactions
0,Bottega,QueFVMcMlT-6aZFv2M47mg,37.754720,-122.421200,"1132 Valencia St, San Francisco, CA 94158",4.3,1210,$$,"(Italian, Pasta Shops, Pizza,)","[pickup, delivery]"
1,Fog Harbor Fish House,f-m7-hyFzkf0HSEeQ2s-9A,37.808673,-122.409821,"39 Pier, San Francisco, CA 94133",4.4,10946,$$,"(Seafood, Wine Bars, Cocktail Bars,)",[restaurant_reservation]
2,Marufuku Ramen,HHtpR0RslupSQ99GIIwW5A,37.785220,-122.431570,"1581 Webster St, Ste 235, San Francisco, CA 94115",4.5,5279,$$,"(Ramen,)","[pickup, delivery]"
3,Blind Butcher,ZSzXw0NgJTyOzcHwKY5eMA,37.761160,-122.434140,"4058 18th St, San Francisco, CA 94114",4.0,407,$$,"(New American, Wine Bars,)",[delivery]
4,Movida,HqTJ1kG1e-HHv3Bx5jHbjA,37.782490,-122.392660,"555 2nd St, San Francisco, CA 94107",4.3,195,N/A,"(Persian/Iranian, Mexican,)",[]
...,...,...,...,...,...,...,...,...,...,...
235,Rich Table,Xdx2_n3nk9Td_aSe7E5ARQ,37.774850,-122.422840,"199 Gough St, San Francisco, CA 94102",4.2,1611,$$$,"(New American, Cocktail Bars, Wine Bars,)",[delivery]
236,Wooden Spoon,UYNswbdsQ5OvwnyY9esxCw,37.766618,-122.430263,"2172 Market St, San Francisco, CA 94114",4.4,744,$$,"(Breakfast & Brunch,)",[delivery]
237,Amarena,SIlvau33Tsp5IuEEmGf46A,37.797682,-122.420361,"2162 Larkin St, San Francisco, CA 94109",4.0,703,$$,"(Italian,)","[pickup, restaurant_reservation, delivery]"
238,Bocado,nS2WlSB9oTM1FZjM_-hcOA,37.783444,-122.399098,"701 Folsom St, San Francisco, CA 94107",4.4,180,N/A,"(Peruvian, Desserts, Cocktail Bars,)","[pickup, restaurant_reservation, delivery]"


Extracting and Transforming Yelp Data

In [42]:
yelp_pd.dtypes

Name             object
Business_ID      object
Latitude        float64
Longitude       float64
Address          object
Rating          float64
Review_count      int64
Price            object
Categories       object
Transactions     object
dtype: object

In [63]:
yelp_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Name          240 non-null    object 
 1   Business_ID   240 non-null    object 
 2   Latitude      240 non-null    float64
 3   Longitude     240 non-null    float64
 4   Address       240 non-null    object 
 5   Rating        240 non-null    float64
 6   Review_count  240 non-null    int64  
 7   Price         240 non-null    object 
 8   Categories    240 non-null    object 
 9   Transactions  240 non-null    object 
dtypes: float64(3), int64(1), object(6)
memory usage: 18.9+ KB


In [69]:
# Transform Yelp data and convert to CSV 
# Take ZIP codes out of address 
yelp_pd['ZIP'] = yelp_pd['Address'].str.extract(r'(\d{5})')

# Convert category column 'Transactions' into three boolean columns
yelp_pd['Pickup'] = yelp_pd['Transactions'].apply(lambda x: 'pickup' in x)
yelp_pd['Delivery'] = yelp_pd['Transactions'].apply(lambda x: 'delivery' in x)
yelp_pd['Restaurant_Reservation'] = yelp_pd['Transactions'].apply(lambda x: 'restaurant_reservation' in x)

# convert to CSV 
yelp_pd.to_csv("CSV_Outputs/Businesses.csv")

yelp_pd.head(5)


,Name,Business_ID,Latitude,Longitude,Address,Rating,Review_count,Price,Categories,Transactions,ZIP,Pickup,Delivery,Restaurant_Reservation
0,Bottega,QueFVMcMlT-6aZFv2M47mg,37.754720,-122.421200,"1132 Valencia St, San Francisco, CA 94158",4.3,1210,$$,"(Italian, Pasta Shops, Pizza,)","[pickup, delivery]",94158,True,True,False
1,Fog Harbor Fish House,f-m7-hyFzkf0HSEeQ2s-9A,37.808673,-122.409821,"39 Pier, San Francisco, CA 94133",4.4,10946,$$,"(Seafood, Wine Bars, Cocktail Bars,)",[restaurant_reservation],94133,False,False,True
2,Marufuku Ramen,HHtpR0RslupSQ99GIIwW5A,37.785220,-122.431570,"1581 Webster St, Ste 235, San Francisco, CA 94115",4.5,5279,$$,"(Ramen,)","[pickup, delivery]",94115,True,True,False
3,Blind Butcher,ZSzXw0NgJTyOzcHwKY5eMA,37.761160,-122.434140,"4058 18th St, San Francisco, CA 94114",4.0,407,$$,"(New American, Wine Bars,)",[delivery],94114,False,True,False
4,Movida,HqTJ1kG1e-HHv3Bx5jHbjA,37.782490,-122.392660,"555 2nd St, San Francisco, CA 94107",4.3,195,N/A,"(Persian/Iranian, Mexican,)",[],94107,False,False,False


Pulling Categories from Yelp 

In [117]:
#TODO # Pull unique categories from Yelp_pd['Categories'] Column and create new table with those categories and IDs 
# Category_ID Category_Name 
categories_copy = yelp_pd['Categories'].copy()

# Step 2: Remove parentheses and split by ', ' to create a list of categories
categories_copy = categories_copy.apply(lambda x: x.strip('()').split(', '))

# Step 3: Flatten the list of all categories
all_categories = [category for sublist in categories_copy for category in sublist if category]

# Step 4: Get unique categories
unique_categories = set(all_categories)

# Display the unique categories
print(unique_categories)

AttributeError: 'tuple' object has no attribute 'strip'

MapBox API Call and Directions Estimate Loop 

In [48]:
#MapBox API 

#Loop for driving
#base url for mapbox 
mapbox_url = "https://api.mapbox.com/directions/v5/mapbox/"
st_coordinates = (37.787937, -122.407677) #Latitude, Longitude in Union Square, SF 
method = "driving"
driving_data = []

for row in yelp_data:
    bs_coordinates = (row['Longitude'], row['Latitude'])
    
    coordinates = f"{st_coordinates[1]},{st_coordinates[0]};{row["Longitude"]},{row["Latitude"]}"
    
    query_url = f"{mapbox_url}{method}/{coordinates}.json"

    #Mapbox paramaters 
    driving_params = {
        "access_token" : mapbox_api_key,
        "overview" : "full",
        # "annotations": "distance, duration"
    }
    
    #Get request for MapBox API 
    mapbox_response = requests.get(query_url, params = driving_params)
    # print(mapbox_response.json()) 
    #if successful, extract duration and distance 
    if mapbox_response.status_code == 200: 
        mapbox_data = mapbox_response.json()

        if 'routes' in mapbox_data and len(mapbox_data['routes']) > 0:
            distance = mapbox_data['routes'][0]['distance']  # Distance in meters
            duration = mapbox_data['routes'][0]['duration']  # Duration in seconds
            route_id = mapbox_data['uuid'] # route_id
            
            driving_data.append({
                "Route_ID": route_id,
                "Distance_Meters": distance, 
                "Duration_Seconds" : duration,
                "Longitude": bs_coordinates[1],
                "Latitude": bs_coordinates[0]
            })
            
        else:
            print(f"Error with MapBox API for {row['name']}: {mapbox_response.status_code}")

driving_pd = pd.DataFrame(driving_data)
driving_pd


,Route_ID,Distance_Meters,Duration_Seconds,Longitude,Latitude
0,asZ1EhTes2skn4adecRzBwpq_hVUjqbJpDeUCkd6UApOgZ...,5125.704,1256.260,37.754720,-122.421200
1,Porj87VvNC7QjMAYyQCAFBpiMpwM_ami-C0yqqfSePIPZi...,3392.329,899.855,37.808673,-122.409821
2,pdMla-P2ZdzjKRTL14NFK-dx6MWuV5uVpeW7UtE8KgIfm0...,2232.918,609.123,37.785220,-122.431570
3,XU_9rq1bDJPq6kVXqih5zxcJarJDvuf6aeju97hS-8icLK...,4746.401,1227.057,37.761160,-122.434140
4,OwEkAt3G5gsbxgSZyD3dRxjVFjLm8VUYNc6FDDFl75qULy...,1976.296,618.342,37.782490,-122.392660
...,...,...,...,...,...
235,LTuIno2zUfV97UONBFCp08bb0Xm5zWH6MKc7h8DyjEyETl...,3149.159,840.571,37.774850,-122.422840
236,M2eP0JzPVxxt_XsJF9cisI5pLOSldhDTYUf6H7SYLpQkgP...,3811.194,961.893,37.766618,-122.430263
237,QVop7QIemsklmFHsZxRiLKwMbYW-I3HR2zHegRY9kzE2FA...,2285.950,603.886,37.797682,-122.420361
238,FLUkS1P7_Ky6bF0d_Rs5GXRI125dnhjRIeqZaIVb9QNUUi...,1509.672,519.069,37.783444,-122.399098


In [34]:
pprint(mapbox_response.json())

{'code': 'Ok',
 'routes': [{'distance': 5833.658,
             'duration': 1297.703,
             'geometry': 'clseFdvbjVXCRCBZJ`BBP?B?@@LFn@N|BLtB@RBRd@tHHfAJzARbDFz@F`ABVJ~AHpAJxAB`@N`CHjAF|@XlEFz@F|@TlDJ|AXxEHhA@L@RBV@PHlAFv@Bh@JxA@TBVLfB@T@V?TAXAP?NA\\AXAV?R@T?N?FBV?N@DXxEXhEJxA@P??BP@TXhE@LFhA@J??Dz@LpBF~@@N@N@L@N@DB\\@R?@b@rGDx@X~DHtA?@BT@N?@JrBt@pK?@Dn@@VBZ@PBP?DBRBR@H@B@DDPFPFRFRFRDRDP@H@F@TBV?@@P@N@NBRd@pHRbDH`AHlA@R@VDd@LpB?f@@X@X@\\B^`@`G@RBVBLBPDR@NDr@BPBLBNFx@@PDTBD@FLVLFRHRDPBNDN@D@H?N@LAT?HAJAZGPCPCn@Iz@Kt@KpC]LClAM^EHCXCRCvC_@PAPCfDa@jDc@NARC^Et@K~@KNCLABA|@Kf@Gd@GJANENCVEB?|@M|@KRKXEt@InAMTCRCB?XEvBWPCNCvBWZEB?PCNCl@Gn@Iz@KFCFEDCHCzAQrC]tAQLAPA@P?HPdCPrC@PBR?F@RNrBNfC@J@NNADAfC[JANCBRd@xHB\\',
             'legs': [{'admins': [{'iso_3166_1': 'US',
                                   'iso_3166_1_alpha3': 'USA'}],
                       'distance': 5833.658,
                       'duration': 1297.703,
                       'steps': [],
                       'summary': 

In [96]:
# Adding column and rearranging 
driving_pd["Duration_Minutes"] = (driving_pd['Duration_Seconds']/60).round()
driving_pd = driving_pd[['Route_ID', 'Distance_Meters', 'Duration_Seconds', 'Duration_Minutes', 'Latitude', 'Longitude']]

driving_pd.head(5)

# Converting to CSV 
yelp_pd.to_csv("CSV_Outputs/Driving_Estimate.csv")

In [62]:
driving_pd.dtypes

Route_ID             object
Distance_Meters     float64
Duration_Seconds    float64
Duration_Minutes    float64
Latitude            float64
Longitude           float64
dtype: object

In [ ]:
# Does the accessibility of transportation affect restaurants differently depending on the type of cuisine they serve?




In [ ]:
# How does transportation accessibility impact restaurants across different price brackets (high-end vs. budget-friendly)?


In [ ]:
# Quantitatively, how does increasing the number of stops on a train or bus line by 5 affect the surrounding restaurant scene?
